In [ ]:
from graphviz import Digraph

dot = Digraph(comment='The Round Table')

dot.node('A', 'King Arthur')
dot.node('B', 'Sir Bedevere the Wise')
dot.node('L', 'Sir Lancelot the Brave')

dot.edges(['AB', 'AL'])
dot.edge('B', 'L', constraint='false')

dot



In [ ]:
from random import random

def randomize_propability(p, delta):
    p += random() * 2* delta - delta
    if p < 0:
        p = 0
    if p > 1:
        p = 1
    return p

In [ ]:
from collections import namedtuple
from random import random
from uuid import uuid4


_P_CLONE_DELTA = 0.1

_P_CONNECTIONS_DELTA = 0.1

_P_CONNECTIONS_CHANGE_LEVEL = 0.3

Node = namedtuple("Node", ("id", "p_clone", "p_connections", "connections"))

def build_connections(connections, p_connections, connected):
    new_connections = []
    if p_connections:
        p = p_connections[0]
        p_connections = p_connections[1:]
        for node in connections:
            if node.id not in connected and random() < p:
                new_connections.append(node)
                connected.add(node.id)
                new_connections.extend(build_connections(node.connections, p_connections, connected))
    return new_connections
    

def evolve(nodes):
    new_nodes = []
    for node in nodes:
        if random() < node.p_clone:
            p_connections = list(node.p_connections)
            if random() < _P_CONNECTIONS_CHANGE_LEVEL:
                len_change = 1 if random() > 0.5 else -1
                if len_change == -1 and len(p_connections) > 1:
                    p_connections = p_connections[:-1]
                elif len_change == 1:
                    p_connections = p_connections + [p_connections[-1]]
            p_connections = [randomize_propability(p, _P_CONNECTIONS_DELTA) for p in p_connections]
            connected = set()
            connections = build_connections([node], p_connections, connected)
            new_nodes.append(
                Node(uuid4(), randomize_propability(node.p_clone, _P_CLONE_DELTA), p_connections, connections))
    return new_nodes


In [ ]:
nodes = [Node(uuid4(), 0.5, [0.5], [])]

In [ ]:
nodes.extend(evolve(nodes))
nodes

In [ ]:
from graphviz import Digraph
from itertools import chain

def find_edges(node, visited_nodes):
    edges = []
    for c in node.connections:
        edges.append((node.id, c.id))
        if c.id not in visited_nodes:
            visited_nodes.append(c.id)
            edges.extend(find_edges(c, nodes))
    return edges

visited_nodes = set((n.id for n in nodes))
edges = chain.from_iterable([find_edges(n, visited_nodes) for n in nodes])

dot = Digraph()
for a, b in edges:
    dot.edge(str(a)[:5], str(b)[:5])
dot